In [1]:
import xarray as xr
import numpy as np
import zarr
import os, sys, time, glob, re

import tensorflow as tf
print(tf.version)
from tensorflow import keras
print(keras.__version__)

from keras import layers
from keras.layers import Layer
from keras import models, losses
from keras.regularizers import l1,l2
from keras.optimizers import Optimizer
# Set the logging level to suppress warnings
tf.get_logger().setLevel(tf.compat.v1.logging.ERROR)

2024-09-18 14:08:51.016071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 14:08:51.027407: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 14:08:51.030914: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-18 14:08:51.039288: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<module 'tensorflow._api.v2.version' from '/home/harish/miniconda3/envs/gUstNET/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>
3.5.0


In [6]:
# Checking if the GPU is available and if it is being used
def run_on_gpu(device_index):
    device_name = f'/device:GPU:{device_index}'
    with tf.device(device_name):
        a = tf.random.uniform([2, 2])
        b = tf.random.uniform([2, 2])
        c = tf.matmul(a, b)
        print(f"Running on {device_name}:")
        print(c.numpy())

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')

# Run the function for each GPU
for i, gpu in enumerate(gpus):
    run_on_gpu(i)

Running on /device:GPU:0:
[[0.36967012 0.6421738 ]
 [0.54994255 0.825116  ]]
Running on /device:GPU:1:
[[0.2043962  0.04633751]
 [0.46446294 0.155029  ]]
Running on /device:GPU:2:
[[0.01552969 0.14567018]
 [0.05410055 0.31523645]]


In [16]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices('GPU')
print(' gpus = ', gpus)
num_gpus = len(gpus)
# Define GPUs
devices = [f'/device:GPU:{i}' for i in range(num_gpus)]

for i, device in enumerate(devices):
    with tf.device(device):
        # Perform training on the assigned GPU
        print(f'Training on device: {device}')

Num GPUs Available:  3
 gpus =  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
Training on device: /device:GPU:0
Training on device: /device:GPU:1
Training on device: /device:GPU:2


In [13]:
#----------------------------------------------------------------
# Build base U-net architecture
#----------------------------------------------------------------
# Non-linear Activation switch - 0: linear; 1: non-linear
# If non-linear, then Leaky ReLU Activation function with alpha value as input
def actv_swtch(swtch, alpha_val):
    if swtch == 0:
        actv = "linear"
    else:
        actv = layers.LeakyReLU(negative_slope=alpha_val)
    return actv

class ReflectPadding2D(Layer):
    def __init__(self, pad_size, **kwargs):
        self.pad_size = pad_size
        super(ReflectPadding2D, self).__init__(**kwargs)

    def call(self, inputs):
        return tf.pad(inputs, [[0, 0], [self.pad_size, self.pad_size], [self.pad_size, self.pad_size], [0, 0]], mode='REFLECT')

    def get_config(self):
        config = super(ReflectPadding2D, self).get_config()
        config.update({"pad_size": self.pad_size})
        return config
# Defining a generic 3D convolution layer for our use
# Inputs: [input tensor, output feature maps, filter size, dilation rate, stride,
# activation switch, if actv switch is 1 then activation-LReLU-alpha value, 
# regularizer factor]
def con2d(inp, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val):
    # Manually apply reflect padding
    pad_size = fil // 2
    # Use ReflectPadding2D instead of custom padding directly
    inp_padded = ReflectPadding2D(pad_size)(inp)
    # Apply the convolutional layer with 'valid' padding since we've already padded the input
    return layers.Conv2D(n_out, (fil, fil), dilation_rate=(dil_rate, dil_rate),
                                  strides=std,
                                  activation=actv_swtch(swtch, alpha_val),
                                  padding="valid", #padding="same",
                                  use_bias=True,
                                  kernel_regularizer=l2(reg_val),
                                  bias_regularizer=l2(reg_val))(inp_padded)

# Residual convolution block
def Res_conv_block(inp, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val):

    y = con2d(inp, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)
    y = con2d(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)
    # Residual connection
    y = layers.Add()([y, con2d(inp, n_out, 1, dil_rate, std, swtch, alpha_val, reg_val)])

    return y

# Convolution downsampling block
def Conv_down_block(inp, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val):

    # Downsampling using the stride 2
    y = con2d(inp, n_out, fil, dil_rate, 2, swtch, alpha_val, reg_val)
    y = con2d(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)
    y = con2d(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)

    return y

# Attention block
def Attention(inp, num_heads, key_dim):

    layer = layers.MultiHeadAttention(num_heads, key_dim, attention_axes=None)
    y = layer(inp, inp)

    return y

# Convolution upsampling block using bilinear interpolation
def Conv_up_block(inp, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val):

    # Upsampling using the stride 2 with transpose convolution
    y = layers.UpSampling2D(size=(2, 2), interpolation='bilinear')(inp)
    y = con2d(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)
    y = con2d(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)

    return y

#2D Pooling layer - inputs[0:avg;1:max, input tensor, pool size]
def pool2d(i, inp, ps):
    if i == 0:
        return layers.AveragePooling2D(pool_size=(ps, ps),
                                            strides=None,
                                            padding='same',
                                            data_format=None)(inp)
    else:
        return layers.MaxPooling2D(pool_size=(ps, ps),
                                            strides=None,
                                            padding='same',
                                            data_format=None)(inp)


# Generator architecture
def Gen(inp_lat, inp_lon, out_lat, out_lon, chnl, out_vars, fil, dil_rate, std, swtch, alpha_val, reg_val, num_heads, key_dim):

    inp = layers.Input(shape=(inp_lat, inp_lon, chnl))
    y_st = layers.Input(shape=(out_lat, out_lon, 2))
    y = y_st

    # Encoding path
    skips = []
    for n_out in [64, 128, 256]:
        y = Res_conv_block(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)
        skips.append(Res_conv_block(y, n_out // 4, fil, dil_rate, std, swtch, alpha_val, reg_val))
        y = Conv_down_block(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)

    # Attention block
    y = Res_conv_block(y, 256, fil, dil_rate, std, swtch, alpha_val, reg_val)
    y = Attention(y, num_heads, key_dim)
    y = Res_conv_block(y, 256, fil, dil_rate, std, swtch, alpha_val, reg_val)
    y = Attention(y, num_heads, key_dim)
    y = Res_conv_block(y, 256, fil, dil_rate, std, swtch, alpha_val, reg_val)

    # Decoding path
    for i, n_out in enumerate([256, 128, 64]):
        y = Conv_up_block(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)
        y = layers.Concatenate(axis=-1)([y, skips[-(i + 1)]])
        y = Res_conv_block(y, n_out, fil, dil_rate, std, swtch, alpha_val, reg_val)

    y = Res_conv_block(y, 32, fil, dil_rate, std, swtch, alpha_val, reg_val)

    y = con2d(y, 32, 1, dil_rate, std, 0, 0, reg_val)
    y = con2d(y, out_vars, 1, dil_rate, std, 0, 0, reg_val)

    return models.Model(inputs=[inp, y_st], outputs=y)

In [14]:
# -------------------------------------------------------------------------
# Model initialising and summary print 
# -------------------------------------------------------------------------
# Build the generator model
x_train = np.random.rand(50,256, 384, 6)
y_train = np.random.rand(50,256, 384, 1)

generator = Gen(x_train.shape[1], x_train.shape[2], y_train.shape[1], y_train.shape[2], 
                  x_train.shape[3], y_train.shape[3], 5, 1, 1, 1, 0.2, 0, 1, 64)
print(generator.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 256, 384,  │          0 │ -                 │
│ (InputLayer)        │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflect_padding2d_… │ (None, 260, 388,  │          0 │ input_layer_13[0… │
│ (ReflectPadding2D)  │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_185 (Conv2D) │ (None, 256, 384,  │      3,264 │ reflect_padding2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflect_padding2d_… │ (None, 260, 388,  │          0 │ conv2d_185[0][0]  │
│ (ReflectPadding2D)  │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflect_padding2d_… │ (None, 256, 384,  │          0 │ input_layer_13[0… │
│ (ReflectPadding2D)  │ 2)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_186 (Conv2D) │ (None, 256, 384,  │    102,464 │ reflect_padding2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_187 (Conv2D) │ (None, 256, 384,  │        192 │ reflect_padding2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_43 (Add)        │ (None, 256, 384,  │          0 │ conv2d_186[0][0], │
│                     │ 64)               │            │ conv2d_187[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflect_padding2d_… │ (None, 260, 388,  │          0 │ add_43[0][0]      │
│ (ReflectPadding2D)  │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_191 (Conv2D) │ (None, 128, 192,  │    102,464 │ reflect_padding2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflect_padding2d_… │ (None, 132, 196,  │          0 │ conv2d_191[0][0]  │
│ (ReflectPadding2D)  │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_192 (Conv2D) │ (None, 128, 192,  │    102,464 │ reflect_padding2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflect_padding2d_… │ (None, 132, 196,  │          0 │ conv2d_192[0][0]  │
│ (ReflectPadding2D)  │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_193 (Conv2D) │ (None, 128, 192,  │    102,464 │ reflect_padding2… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflect_padding2d_… │ (None, 132, 196,  │          0 │ conv2d_193[0][0]  │
│ (ReflectPadding2D)  │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_194 (Conv2D) │ (None, 128, 192,  │    204,928 │ reflect_padding2… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflect_padding2d_… │ (None, 132, 196,  │          0 │ conv2d_194[0][0]

 Total params: 30,368,689 (115.85 MB)

 Trainable params: 30,368,689 (115.85 MB)

 Non-trainable params: 0 (0.00 B)

None
